In [1]:
vcf_file = 'arp-v2.1-mc-chm13.vcf'


In [16]:
import pickle

In [2]:
def extract_info(info_str, key):
    for field in info_str.split(';'):
        if field.startswith(key + '='):
            return field[len(key)+1:]
    return None

In [3]:
def allele_lengths(ref, alt):
    alts = alt.split(',')
    return [len(a) for a in alts]

In [4]:
interesting_alleles = []
with open(vcf_file, 'r') as vcf:
    for line in vcf:
        if not line.startswith('#'):  # Exclude header lines
            columns = line.strip().split('\t')
            ref = columns[3]
            alt = columns[4]
            
            lengths = allele_lengths(ref, alt)

            if len(lengths) >= 5 and any((l-len(ref)) > 10000 for l in lengths):
                interesting_alleles.append(line.strip())


In [7]:
interesting_alleles[0][:10]

'chr1\t20966'

In [8]:
locations = [(allele.split()[0], allele.split()[1]) for allele in interesting_alleles]

In [13]:
regions = [(region[0], int(region[1] )- 10000, int(region[1]) + 10000) for region in locations]

In [18]:
with open('locations.dump', 'wb') as f:
    pickle.dump(regions, f)

In [14]:
pd.DataFrame('')

[('chr1', 2086666, 2106666),
 ('chr1', 2092619, 2112619),
 ('chr1', 2092916, 2112916),
 ('chr1', 2150088, 2170088),
 ('chr1', 5522193, 5542193),
 ('chr1', 7037051, 7057051),
 ('chr1', 12660623, 12680623),
 ('chr1', 12758473, 12778473),
 ('chr1', 16287070, 16307070),
 ('chr1', 16306941, 16326941),
 ('chr1', 16430888, 16450888),
 ('chr1', 16642378, 16662378),
 ('chr1', 31274140, 31294140),
 ('chr1', 38834125, 38854125),
 ('chr1', 54384011, 54404011),
 ('chr1', 72169591, 72189591),
 ('chr1', 75210381, 75230381),
 ('chr1', 109702345, 109722345),
 ('chr1', 109702373, 109722373),
 ('chr1', 120828231, 120848231),
 ('chr1', 120834434, 120854434),
 ('chr1', 121617957, 121637957),
 ('chr1', 121761172, 121781172),
 ('chr1', 121820186, 121840186),
 ('chr1', 126260773, 126280773),
 ('chr1', 126610069, 126630069),
 ('chr1', 126631704, 126651704),
 ('chr1', 126919402, 126939402),
 ('chr1', 126931687, 126951687),
 ('chr1', 126943307, 126963307),
 ('chr1', 127054723, 127074723),
 ('chr1', 127117805, 12

In [18]:
with open('interesting_alleles.vcf', 'w') as f:
    for alleles in interesting_alleles:
        f.write(alleles + '\n')

In [19]:
def find_max_per_chrom(sites):
    max = {}
    for site in sites:
        chrom, loc = site.split()[0], int(site.split()[1])
        
        if loc > max.get(chrom, 0):
            max[chrom] = loc
    
    
    return max


In [15]:
import pandas as pd

In [21]:
def get_df(sites):
    data = []
    for site in sites:
        data.append((site.split()[0], int(site.split()[1])))
    
    return pd.DataFrame(data, columns=['chrom', 'pos'])

In [22]:
site_df = get_df(interesting_alleles)

In [23]:
len(interesting_alleles)

715

In [24]:
window_size = 100000
regions = []
for chrom in site_df['chrom'].unique():
    temp_df = site_df.loc[site_df['chrom'] == chrom]
    max_position = temp_df['pos'].max()
    min_position = temp_df['pos'].min()
    
    ptr = min_position
    end = max_position
    
    
    
    while ptr < end:
        sites_in_window = temp_df.loc[(temp_df['pos']>ptr) & (temp_df['pos']<ptr+window_size)]
        if sites_in_window.shape[0] >= 2:
            regions.append((chrom, ptr, ptr+window_size))
            
        ptr += window_size
    
    
        

In [25]:
len(regions)

113

In [26]:
def variants_within_flank(variants, genes, start, end):
    result = {}
    
    for i,line in variants.iterrows():
        chrom = line['CHROM']
        pos = int(line['POS'])
        start_flank = start
        end_flank = end
        
        # Check if genes are within the flanking region
        for start, end, gene_name in genes.get(chrom, []):
                    if start < end_flank and end > start_flank:
                        key = (chrom, pos)
                        if key not in result:
                            result[key] = []
                        result[key].append(gene_name)
    return result

In [27]:
pd.DataFrame(regions, columns=['columns', 'start', 'end']).to_csv('regions_100kb_window.csv')

In [28]:
def find_genes_in_regions(gff3_file, regions):
    """
    Find genes in a GFF3 file that lie entirely or partially within the given regions.

    gff3_file: Path to the GFF3 file
    regions: List of tuples representing regions as (chromosome, start, end)
    """
    genes_in_regions = {}

    with open(gff3_file, 'r') as file:
        # Filter GFF3 lines to consider only gene entries
        gene_lines = [line.strip() for line in file.readlines() if 'gene' in line ]

    # For each region, check which genes lie within or overlap it
    for region in regions:
        chrom, start, end = region
        genes_in_current_region = []

        for line in gene_lines:
            print(line)
            fields = line.split('\t')
            gene_chrom, gene_start, gene_end = fields[0], int(fields[3]), int(fields[4])
            gene_id = [item for item in fields[8].split(';') if 'ID=' in item][0].split('=')[1]

            # Check for overlap: conditions adjusted to allow partial overlap
            if gene_chrom == chrom and not (gene_end < start or gene_start > end):
                genes_in_current_region.append(gene_id)

        genes_in_regions[region] = genes_in_current_region

    return genes_in_regions

In [32]:
def find_entries_in_regions(gff3_file, regions):
    """
    Find all entries in a GFF3 file that lie entirely or partially within the given regions.

    gff3_file: Path to the GFF3 file
    regions: List of tuples representing regions as (chromosome, start, end)
    """
    entries_in_regions = {}

    with open(gff3_file, 'r') as file:
        # Skip header lines (lines starting with ##)
        entries = [line.strip() for line in file.readlines() if not line.startswith('##')]

    for region in regions:
        chrom, start, end = region
        entries_in_current_region = []

        for line in entries:
            fields = line.split('\t')
            entry_chrom, entry_start, entry_end = fields[0], int(fields[3]), int(fields[4])
            entry_type = fields[2]
            entry_info = fields[8]

            if entry_chrom == chrom and not (entry_end < start or entry_start > end):
                # Append the type of the entry and its info for labeling
                entries_in_current_region.append(f"{entry_type}: {entry_info}")

        entries_in_regions[region] = entries_in_current_region

    return entries_in_regions

In [29]:
gff3_file = 'chm13v2.0_RefSeq_Liftoff_v5.1.gff3'

In [33]:
regional_genes = find_entries_in_regions(gff3_file, regions)

In [22]:
genes_per_loc = []
for k, v in regional_genes.items():
    genes_per_loc.append((k[0],k[1], k[2], ','.join(v)))

In [32]:
pd.DataFrame(genes_per_loc, columns=['chr', 'start', 'end', 'genes']).to_csv('genes_per_window.csv',index=None)

,gene-symbol
0,A2M
1,A2ML1
2,AAAS
3,AAGAB
4,AARS2
...,...
4003,ZP3
4004,ZPBP
4005,ZPR1
4006,ZSWIM6


In [32]:
rows = [(key, val) for key, values in regional_genes.items() for val in values]
gene_df = pd.DataFrame(rows, columns=['loc', 'gene'])


recessive_genes = pd.read_excel('disorder gene list.xlsx')
intersect = gene_df.merge(recessive_genes, how='left', left_on='gene', right_on='gene-symbol').dropna().drop(['gene-symbol'], axis=1)
intersect_dict = {}
for i, row in intersect.iterrows():
    if row['loc'] not in intersect_dict:
        intersect_dict[row['loc']] = [] 
    intersect_dict[row['loc']].append(str(row['gene']))

In [33]:
intersect

,loc,gene
17,"(chr1, 109696666, 109796666)",EPS8L3
29,"(chr1, 160496666, 160596666)",SDHC
200,"(chr3, 131388491, 131488491)",RAB7A
